In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 이미지 불러오기
import os, glob
import numpy as np
from PIL import Image

def load_images_from_folder(folder, target_size=(256, 256)):
    paths = sorted(glob.glob(os.path.join(folder, '*.png')))
    images = []
    for path in paths:
        img = Image.open(path).convert('L').resize(target_size)
        img_array = np.array(img) / 255.0
        images.append(img_array[..., np.newaxis])  # (256,256,1)
    return np.array(images)

image_folder = "drive/MyDrive/Colab Notebooks/Project2/Model/img256"
images = load_images_from_folder(image_folder)
print("Loaded image shape:", images.shape)  # (N, 256, 256, 1)

In [ ]:
# 시퀀스 데이터 생성 (12장 입력 → 13번째 예측)
seq_len = 12
X, y = [], []
for i in range(len(images) - seq_len):
    X.append(images[i:i+seq_len])  # (12, 256, 256, 1)
    y.append(images[i+seq_len])    # (256, 256, 1)
X = np.array(X)
y = np.array(y)
print("X shape:", X.shape, "y shape:", y.shape)

In [ ]:
# Autoencoder 인코더 정의
import tensorflow as tf
from tensorflow.keras import layers, models

encoding_dim = 256

def build_encoder(input_shape=(256, 256, 1), latent_dim=256):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2)(x)  # 128x128
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)  # 64x64
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(2)(x)  # 32x32
    x = layers.Flatten()(x)
    latent = layers.Dense(latent_dim)(x)
    return models.Model(inputs, latent, name='Encoder')


encoder = build_encoder()
print(encoder.summary())

In [ ]:
# AE 구조 학습 후 예측된 latent가 아닌, 진짜 latent로 복원 가능한지 테스트
latent = encoder.predict(images[0:1])
reconstructed = decoder.predict(latent)

plt.imshow(reconstructed[0].squeeze(), cmap='gray')

In [ ]:
from tensorflow.keras import layers, models

def build_decoder(latent_dim=256):
    inputs = layers.Input(shape=(latent_dim,))
    x = layers.Dense(32*32*128, activation='relu')(inputs)
    x = layers.Reshape((32, 32, 128))(x)
    x = layers.Conv2DTranspose(128, 3, strides=2, padding='same', activation='relu')(x)  # 64x64
    x = layers.Conv2DTranspose(64, 3, strides=2, padding='same', activation='relu')(x)   # 128x128
    x = layers.Conv2DTranspose(32, 3, strides=2, padding='same', activation='relu')(x)   # 256x256
    outputs = layers.Conv2D(1, 3, activation='sigmoid', padding='same')(x)
    return models.Model(inputs, outputs, name='Decoder')



In [ ]:
# 모델 연결
encoder = build_encoder()
decoder = build_decoder()
ae_input = layers.Input(shape=(256, 256, 1))
latent = encoder(ae_input)
recon = decoder(latent)
autoencoder = models.Model(ae_input, recon)
autoencoder.compile(optimizer='adam', loss='mse')

# 학습
history = autoencoder.fit(images, images, epochs=50, batch_size=32, validation_split=0.1)


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Autoencoder Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
test_img = images[0:1]
recon_img = autoencoder.predict(test_img)

plt.subplot(1,2,1)
plt.title("Original")
plt.imshow(test_img[0].squeeze(), cmap='gray')

plt.subplot(1,2,2)
plt.title("Reconstructed")
plt.imshow(recon_img[0].squeeze(), cmap='gray')
plt.show()


In [ ]:
# GRU 시계열 예측기 정의
def build_gru_predictor(seq_len, latent_dim):
    model = models.Sequential([
        layers.GRU(128, input_shape=(seq_len, latent_dim)),
        layers.Dense(latent_dim)
    ])
    return model

predictor = build_gru_predictor(seq_len, encoding_dim)
predictor.compile(optimizer='adam', loss='mse')
print(predictor.summary())

In [ ]:
# 이미지 ➝ latent 변환
latent_seqs = []
latent_targets = []
for i in range(len(X)):
    latent_seq = [encoder(img[np.newaxis, ...])[0].numpy() for img in X[i]]
    latent_seqs.append(latent_seq)
    latent_target = encoder(y[i][np.newaxis, ...])[0].numpy()
    latent_targets.append(latent_target)

latent_seqs = np.array(latent_seqs)
latent_targets = np.array(latent_targets)

print("Latent sequence shape:", latent_seqs.shape)
print("Latent target shape:", latent_targets.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler

# (1) latent_seqs: (num_samples, 12, latent_dim)
# (2) latent_targets: (num_samples, latent_dim)

# reshape for scaling
latent_seqs_reshaped = latent_seqs.reshape(-1, latent_seqs.shape[-1])
scaler = StandardScaler()
latent_seqs_scaled = scaler.fit_transform(latent_seqs_reshaped).reshape(latent_seqs.shape)

latent_targets_scaled = scaler.transform(latent_targets)


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense

latent_dim = latent_seqs.shape[-1]

predictor = Sequential([
    GRU(128, input_shape=(12, latent_dim)),
    Dropout(0.2),
    Dense(latent_dim)
])
predictor.compile(optimizer='adam', loss='mse')
predictor.summary()

In [ ]:
# 8. GRU 훈련
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history_gru = predictor.fit(latent_seqs_scaled, latent_targets_scaled,
                            epochs=100,
                            batch_size=32,
                            validation_split=0.1,
                            callbacks=[early_stop])

In [ ]:
plt.plot(history_gru.history['loss'], label='train_loss')
plt.plot(history_gru.history['val_loss'], label='val_loss')
plt.title('GRU Predictor Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# === 예측 및 복원 ===

idx = 0  # 테스트할 시퀀스 인덱스

# predicted_latent = predictor.predict(latent_seqs_scaled[idx:idx+1])
# predicted_image = decoder.predict(predicted_latent)

# GRU 예측 (정규화된 latent vector)
predicted_latent_scaled = predictor.predict(latent_seqs_scaled[idx:idx+1])

# 역정규화
predicted_latent = scaler.inverse_transform(predicted_latent_scaled)

# 디코더로 이미지 복원
predicted_image = decoder.predict(predicted_latent)

# 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(18,6))
plt.subplot(1,3,1)
plt.title("Ground Truth")
plt.imshow(y[idx].squeeze(), cmap='gray')

plt.subplot(1,3,2)
plt.title("Predicted")
plt.imshow(predicted_image[0].squeeze(), cmap='gray')

plt.subplot(1,3,3)
error_map = np.abs(y[idx].squeeze() - predicted_image[0].squeeze())
mse = np.mean((y[idx] - predicted_image[0])**2)
error_percent = mse * 100
plt.title(f"Error Map\nMSE: {mse:.5f}, Error: {error_percent:.2f}%")
plt.imshow(error_map, cmap='hot')
plt.colorbar()

plt.tight_layout()
plt.show()


In [ ]:
# # === Autoregressive N-step 예측 ===

# # 초기 latent 시퀀스 선택 (정규화된 값 사용)
# input_seq = latent_seqs_scaled[idx:idx+1].copy()  # (1, 12, latent_dim)

# n_future = 5  # 예측할 미래 이미지 수
# generated_latents_scaled = []

# for _ in range(n_future):
#     # GRU 예측 (정규화된 latent vector)
#     pred_scaled = predictor.predict(input_seq)

#     # 저장
#     generated_latents_scaled.append(pred_scaled[0])

#     # 다음 입력 시퀀스로 갱신
#     input_seq = np.concatenate([input_seq[:,1:,:], pred_scaled[:,np.newaxis,:]], axis=1)

# # 역정규화
# generated_latents = scaler.inverse_transform(np.array(generated_latents_scaled))  # (n_future, latent_dim)

# # 이미지 복원
# generated_images = decoder.predict(generated_latents)

# # 시각화
# import matplotlib.pyplot as plt

# plt.figure(figsize=(15,3))
# for i in range(n_future):
#     plt.subplot(1, n_future, i+1)
#     plt.imshow(generated_images[i].squeeze(), cmap='gray')
#     plt.title(f"T+{i+1}")
#     plt.axis('off')
# plt.tight_layout()
# plt.show()
